This notebook cleans .txt and .dta data for students performance on exams for BIO1111 and exports a csv which can be analyzed in an R script.
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob
from soar import *

pd.options.display.max_columns = 50

# Setting source and output folders

In [2]:
def sourceFiles(filedict,source = 'worksource', output = 'workoutput'):
    filedict = filedict[input('Choose a filesystem: ')]
    filedict[source] = filedict[source].format(year=year,semester=semester,exNum=exNum)
    filedict[output] = filedict[output].format(year=year,semester=semester,exNum=exNum)
    return filedict

In [3]:
sources = {'tu':{'worksource': "C:/Users/craga/Google Drive/TU {year}/Exams/{semester}/Exam {exNum}/Input Data/",
    'workoutput': "C:/Users/tuh27554/Documents/Grading/{year}/{semester} EX{exNum}/Output Data/"},
        'gandolf':{'worksource': "C:/Users/craga/Google Drive/TU {year}/{semester}/Exams/Exam {exNum}/Input Data/",
    'workoutput': "C:/Users/craga/Google Drive/TU {year}/{semester}/Exams/Exam {exNum}/Output Data/"}}

In [4]:
year=input('which year? ')
semester=input('which semester? ')
exNum=input('which exam number? ')

which year? 2020
which semester? Spring
which exam number? 2


In [5]:
sources = sourceFiles(sources)

Choose a filesystem: gandolf


In [6]:
sources

{'worksource': 'C:/Users/craga/Google Drive/TU 2020/Spring/Exams/Exam 2/Input Data/',
 'workoutput': 'C:/Users/craga/Google Drive/TU 2020/Spring/Exams/Exam 2/Output Data/'}

# Getting and Cleaning Exam Data

In [7]:
os.chdir(sources['worksource'])
files = glob.glob('*.dta')
files

['marcexams_2019_2019-A2534.dta']

In [8]:
soars = [int(x) for x in input('Enter your SOAR sessions separated by commas: ').split(',')]
df1=cleanSoarExam(files[0],
                  examNum=input('Which exam number is this? '),
                  soarSessions=soars)
df1['version'] = 1
# examversions = int(input('How many exam versions are there? '))
# if examversions>1:
#     versionList = [x for x in [y+1 for y in range(examversions+1)]]
# df2=cleanSoarExam(files[1],examNum=1,soarSessions=[73,74,75])
# df2['version'] = 2
df = df1.loc[(df1.soar.isin([np.nan]+soars))&(df1.soarType.isin(['key','mine']))]#.append(df2)#remove students not in your sections
df.head()

Enter your SOAR sessions separated by commas: 73,74,75
Which exam number is this? 2


C:\Users\craga\Documents\GitHub\Grading\soar.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tmp['soar'] = soar


,tuid,last,first,middle,unnamed1,unnamed2,unnamed3,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType,version
0,500 - BLU,EXAM - KEY,NNNNNN,N,NaN,NaN,NaN,NaN,27,2,3,2,4,4,4,1,3,1,3,2,2,2,1,4,2,2,3,4,2,2,3,1,1,2,1,2,4,key,1
9,915619341,RANJI,LAUREN,NaN,22.0,21.0,24.0,73.0,9,2,2,4,2,1,2,4,1,1,2,3,2,4,1,3,2,1,3,4,2,4,1,3,1,2,4,4,3,mine,1
10,915767393,QAZI,ZEMAR,NaN,22.0,20.0,9.0,75.0,10,2,3,4,3,4,4,4,2,4,3,3,3,4,1,4,1,2,4,4,1,4,3,2,1,3,3,1,2,mine,1
12,915637331,SATUNAS,KATELY,NaN,22.0,20.0,13.0,75.0,12,2,4,4,3,4,2,4,3,1,1,3,4,2,4,4,3,4,2,4,1,2,3,1,1,2,4,3,4,mine,1
14,914894157,NELMS,MATT,B,22.0,21.0,96.0,73.0,18,2,3,2,4,4,2,1,2,4,3,4,2,2,1,3,2,2,3,2,2,4,2,1,1,2,1,2,3,mine,1


Now we change the dir to the outputfolder and save the cleaned, merged file.

In [9]:
itemcols = df.columns[df.columns.str.contains('item')]
itemcols

Index(['item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7',
       'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13',
       'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19',
       'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25',
       'item_26', 'item_27'],
      dtype='object')

In [10]:
for col in itemcols:
    df[col] = df[col].apply(letters2num)

C:\Users\craga\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
os.chdir(sources['workoutput'])
df.to_csv(input('Enter filename prefix: ')+'.csv')

Enter filename prefix: Spring 2020 Exam 2 Data for my sections


## Extract Student Data

In [12]:
os.chdir(sources[input('Which location in sources should be used? ')])
examData = pd.read_csv(input('Which file would you like to use? ')+'.csv')
examData.head()

Which location in sources should be used? workoutput
Which file would you like to use? Spring 2020 Exam 2 Data for my sections


,Unnamed: 0,tuid,last,first,middle,unnamed1,unnamed2,unnamed3,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType,version
0,0,500 - BLU,EXAM - KEY,NNNNNN,N,NaN,NaN,NaN,NaN,27,2,B,C,A,A,A,D,B,D,B,C,C,C,D,A,C,C,B,A,C,C,B,D,D,C,D,C,A,key,1
1,9,915619341,RANJI,LAUREN,NaN,22.0,21.0,24.0,73.0,9,2,C,A,C,D,C,A,D,D,C,B,C,A,D,B,C,D,B,A,C,A,D,B,D,C,A,A,B,mine,1
2,10,915767393,QAZI,ZEMAR,NaN,22.0,20.0,9.0,75.0,10,2,B,A,B,A,A,A,C,A,B,B,B,A,D,A,D,C,A,A,D,A,B,C,D,B,B,D,C,mine,1
3,12,915637331,SATUNAS,KATELY,NaN,22.0,20.0,13.0,75.0,12,2,A,A,B,A,C,A,B,D,D,B,A,C,A,A,B,A,C,A,D,C,B,D,D,C,A,B,A,mine,1
4,14,914894157,NELMS,MATT,B,22.0,21.0,96.0,73.0,18,2,B,C,A,A,C,D,C,A,B,A,C,C,D,B,C,C,B,C,C,A,C,D,D,C,D,C,B,mine,1


In [15]:
studentids = input('Provide student IDs separated ids by commas.').split(',')
exportData = pd.DataFrame()
if len(studentids[0])==0:
    lnames = input('Provide student last names separated by commas.').split(',')
    fnames = input('Provide student first names separated by commas.').split(',')
    namelist = [val for val in [lnames,fnames] if len(val)>0]
    if len(namelist)==0:
        print('No identifying information provided.  Data will not be filtered.')
    else:
        if len(lnames[0])>0:
            exportData = examData.loc[examData['last'].isin(lnames)]
        if len(fnames[0])>0:
            try:
                exportData = exportData.merge(examData.loc[examData['first'].isin(fnames)],
                                          how = 'inner')
            except:
                print('No last names provided.')
                exportData = exportData\
                .append(examData.loc[examData['first'].isin(fnames)],sort=True)
        if len(namelist)==1:
            print('Limited identifying information provided.  Filtered data should be checked\
            to ensure only appropriate student data is being provided prior to delivering files.')
            
else:
    exportData = examData.loc[examData.tuid.isin(studentids)]
year = input('Year?')
semester = input('Semester?')
exam = input('Exam Number?')
for student in exportData.tuid.unique():
    tmp = examData.loc[(examData.soarType=='key')&(examData.soar.isna())]\
    .append(exportData.loc[examData.tuid==student]).drop_duplicates()\
    .drop(columns = ['version']+[col for col in examData.columns if 'nnamed' in col]).to_csv('{}_{}_{}_{}.csv'.format(year,semester,exam,student),index=False)
exportData.drop(columns = ['version']+[col for col in examData.columns if 'nnamed' in col])

Provide student IDs separated ids by commas.915697241
Year?2020
Semester?Spring
Exam Number?2


,tuid,last,first,middle,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType
31,915697241,FABBRI,KATHRY,E,75.0,13,2,A,A,B,A,A,A,B,A,B,C,B,C,A,A,B,D,B,A,C,C,D,B,D,B,A,D,A,mine


In [14]:
13/25*100

52.0

In [ ]:
studentids = input('Provide student IDs separated ids by commas.').split(',')
def exportStudent
exportData = pd.DataFrame()
if len(studentids[0])==0:
    lnames = input('Provide student last names separated by commas.').split(',')
    fnames = input('Provide student first names separated by commas.').split(',')
    namelist = [val for val in [lnames,fnames] if len(val)>0]
    if len(namelist)==0:
        print('No identifying information provided.  Data will not be filtered.')
    else:
        if len(lnames[0])>0:
            exportData = examData.loc[examData['last'].isin(lnames)]
        if len(fnames[0])>0:
            try:
                exportData = exportData.merge(examData.loc[examData['first'].isin(fnames)],
                                          how = 'inner')
            except:
                print('No last names provided.')
                exportData = exportData\
                .append(examData.loc[examData['first'].isin(fnames)],sort=True)
        if len(namelist)==1:
            print('Limited identifying information provided.  Filtered data should be checked\
            to ensure only appropriate student data is being provided prior to delivering files.')
            
else:
    exportData = examData.loc[examData.tuid.isin(studentids)]
year = input('Year?')
semester = input('Semester?')
exam = input('Exam Number?')
for student in exportData.tuid.unique():
    tmp = examData.loc[(examData.soarType=='key')&(examData.soar.isna())]\
    .append(exportData.loc[examData.tuid==student]).drop_duplicates()\
    .to_csv('{}_{}_{}_{}.csv'.format(year,semester,exam,student),index=False)
exportData

# Analyzing Exam Data

1. [Most Frequently Wrong](#Most-Frequently-Wrong-Answers)

## Strange values for items

In reviewing the data I noted some values outside of the expected 1-4.  Could you help me understand what these are?

In [ ]:
all_responses = pd.concat([pd.Series(df[col].unique()) for col in itemcols]).unique()
strange_response = [resp for resp in all_responses if resp not in ['A','B','C','D','E','1','2','3','4','5']]
strange_resp_item=[col for col in itemcols if df.loc[df[col].isin(strange_response),col].shape[0]>0]
print("The strange responses {} were found for the following items:\n{}".format(strange_response, strange_resp_item))


## Most Frequently Wrong Answers

Here we identify the columns containing information about responses.

In [ ]:
df.soarType.value_counts()

Now we apply this function to create a ranked list of items by order of the number of my students getting them wrong.

In [ ]:
wrongcount_v1_mine = pd.DataFrame()
for soar in [x for x in df.soar.unique() if not np.isnan(x)]:
    for col in itemcols:
        wrongcount_v1_mine = wrongcount_v1_mine.append(countwrong(df= df.loc[df.soar==soar],
                                                                  col = col, 
                                                                  ver = 1,
                                                                  compind=['mine','other']))
    wrongcount_v1_mine = wrongcount_v1_mine.sort_values(['number','item'],
                                                        ascending=False)\
    .reset_index(drop = True)
    # wrongcount_v1_mine.index = wrongcount_v1_mine.item
    # wrongcount_v1_mine= wrongcount_v1_mine.drop(columns = 'item')
    print('wrongcount_{}:\n{}'.format(int(soar),wrongcount_v1_mine))
    total_wrongtmp = wrongcount_v1_mine.set_index(['item']).sum(axis=1)
    total_wrongtmp.to_csv('total wrong {}.csv'.format(int(soar)),index = True)
    print('total_wrong_{}:\n{}'.format(int(soar),
                                       total_wrongtmp.sort_values(ascending=False)))

## Printing the Keys

In [ ]:
# import numpy as np
v1keys = df.loc[(df.soar.isnull())&(df.soarType=='key')&(df.version==1),itemcols].iloc[0]
# v2keys = df.loc[(df.soarType=='key')&(df.version==2),itemcols].iloc[0]
keys = pd.DataFrame(data = {'version1':v1keys})#,'version2':v2keys})
keys.to_csv(input('Enter filename prefix for the keys: ')+'.csv',index = False)
keys

# Evaluating Student Performance

## Score distribution
Now we look at scores by section. 

In [ ]:
descCols= ['mean','min','25%','50%','75%','max']
dfDesc = df.loc[(df.soarType!='key')].groupby(['soarType','soar']).ncorrect.describe()
dfDesc = pd.concat([dfDesc[['count']],dfDesc[descCols].apply(lambda x: x/25*100)],axis=1)
dfDesc

## Item Analysis
1. [Percent choice for each item](#Percent-choice-for-each-item)

os.chdir(source['worksource'])
df20 = pd.read_csv('Spring 2020v1.csv')
dfcomp = pd.read_csv('Cross-Walk 2019 Fall to 2018 Fall Exams.csv')
dfcomp

### Percent choice for each item

df_item1 = pd.DataFrame()
for col in itemcols:
    df_item1 = df_item1.append(df.loc[df.version==1,col].value_counts(normalize=True))
df_item1 = df_item1.applymap(lambda x: "{:.0f}".format(x*100))
df_item1 = df_item1.reset_index().rename(columns={'index':'F19E1'})
df_item1['F19E1'] = df_item1.F19E1.apply(lambda x: int(x[5:]))
df_item1 = df_item1.sort_values('F19E1')
df_item1['F18E1'] = dfcomp.F18E1
df_item1.to_csv('Fall 2019v1.csv')
df_item1

df_item = pd.DataFrame()
for col in itemcols:
    df_item = df_item.append(df.loc[df.version==2,col].value_counts(normalize=True))
df_item = df_item.applymap(lambda x: "{:.0f}".format(x*100))
df_item

df18

dfcomp